In [1]:
from utils import load_data
import pandas as pd
from autogluon.tabular import TabularPredictor

/home/julian/miniconda3/envs/dtiam-esm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = 'kiba'
setting = 'warm_start'
fold = 0
dataset_path = "../data/dta/" + dataset
folds_path = dataset_path + f"/data_folds/{setting}/"

! ls {folds_path}

test_fold_0.csv  test_fold_3.csv   train_fold_1.csv  train_fold_4.csv
test_fold_1.csv  test_fold_4.csv   train_fold_2.csv
test_fold_2.csv  train_fold_0.csv  train_fold_3.csv


In [3]:
import pickle
comp_feat = pickle.load(open(dataset_path + "/features/compound_features.pkl", "rb"))
prot_feat = pickle.load(open(dataset_path + "/features/protein_features.pkl", "rb"))

In [4]:
i = 0
train_data, test_data = load_data(folds_path, i, comp_feat, prot_feat)

Loading data ...


In [5]:
assert train_data.shape[1] == (list(comp_feat.values())[0].shape[0] + list(prot_feat.values())[0].shape[0] + 1)

In [6]:
res_all = pd.DataFrame(columns=["RMSE", "MSE", "Pearson", "Spearman", "CI"])
res_all

,RMSE,MSE,Pearson,Spearman,CI


In [8]:
included_model_types = ['NN_TORCH']
eval_metric = None
preset=None
predictor = TabularPredictor(label="y", eval_metric=eval_metric).fit(
    train_data=train_data, included_model_types=included_model_types, presets=preset
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250616_162458"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Apr 17 19:20:47 UTC 2
CPU Count:          20
Memory Avail:       41.17 GB / 62.63 GB (65.7%)
Disk Space Avail:   231.41 GB / 1684.96 GB (13.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Us

In [9]:
test_data_nolab = test_data.drop(columns=["y"])

In [10]:
pred_scores = predictor.predict(test_data_nolab)

In [11]:
import numpy as np
from utils import load_data, rmse, mse, pearson, spearman, ci, roc_auc, pr_auc

G, P = np.array(test_data["y"]), np.array(pred_scores)
ret = [rmse(G, P), mse(G, P), pearson(G, P), spearman(G, P), ci(G, P)]

print(
    f"fold: {i+1}, RMSE: {ret[0]}, MSE: {ret[1]}, Pearson: {ret[2]}, Spearman: {ret[3]}, CI: {ret[4]}"
)
res_all.loc[i] = ret

fold: 1, RMSE: 0.44492112563551195, MSE: 0.19795480803677099, Pearson: 0.8490002971155794, Spearman: 0.8471892986248705, CI: 0.8745371729063883


In [12]:
res_all

,RMSE,MSE,Pearson,Spearman,CI
0,0.444921,0.197955,0.849,0.847189,0.874537


In [18]:
res_all

,RMSE,MSE,Pearson,Spearman,CI
0,0.39193,0.153609,0.883442,0.878446,0.882292
